We want to calculate the covariance matrix of our background data.



In [36]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [261]:
full_time_series = torch.from_numpy(np.loadtxt("./data/background_0.dat", dtype=np.float32))
full_time_series = time_series.to(device)

In [269]:
time_series = full_time_series[0:10000]

In [270]:
#time_series = torch.normal(10, 1, size=(1, 1000000),dtype=torch.float32)[0]

# sliding window of size 5, rows are the waveform slices
windows = time_series.unfold(0,5,1)

# windows.T is rows of our random variables, take mean per row
means = torch.mean(windows.T, axis=1)

# windows - means subtracts means of each var from each window, so now it is window of differences from mean
cov_mat = (windows - means).T @ (windows - means) * (1.0 / (windows.shape[0]))
#inv_cov_mat = torch.inverse(cov_mat)

print(cov_mat)
#print(inv_cov_mat)

#print(cov_mat @ inv_cov_mat)

tensor([[ 1.0211,  0.0114,  0.0116, -0.0081, -0.0206],
        [ 0.0114,  1.0211,  0.0113,  0.0116, -0.0081],
        [ 0.0116,  0.0113,  1.0211,  0.0112,  0.0115],
        [-0.0081,  0.0116,  0.0112,  1.0212,  0.0113],
        [-0.0206, -0.0081,  0.0115,  0.0113,  1.0212]])


In [271]:
sliding_window_size = 5

# sliding window of size 5, rows are the waveform slices
windows = time_series.unfold(0, sliding_window_size, 1)

# windows.T is rows of our random variables, take mean per row
means = torch.mean(windows.T, axis=1)

cov_row_one = F.conv1d(time_series.view(1, 1, -1), time_series.view(1, 1, -1)[:,:,0:time_series.shape[0] + 1 - sliding_window_size]) / (time_series.shape[0] + 1 - sliding_window_size) - means * means[0]

print(cov_row_one[0,0])

tensor([ 1.0213,  0.0113,  0.0116, -0.0083, -0.0208])


In [222]:
F.conv1d(time_series.view(1, 1, -1), time_series.view(1, 1, -1)[:,:,0:time_series.shape[0] + 1 - sliding_window_size])

tensor([[[ 55.,  70.,  85., 100., 115.]]])

In [232]:
means

tensor([2.5000, 3.5000, 4.5000, 5.5000, 6.5000])

In [231]:
F.conv1d(time_series.view(1, 1, -1), time_series.view(1, 1, -1)[:,:,0:time_series.shape[0] + 1 - sliding_window_size]) / (time_series.shape[0] + 1 - sliding_window_size) - means * means[0]

tensor([[[2.9167, 2.9167, 2.9167, 2.9167, 2.9167]]])

In [242]:
sum = 0

for i in range(0,6):
    print((i - 2.5) * (i + 1 - 3.5))
    sum += (i - 2.5) * (i + 1 - 3.5)

6.25
2.25
0.25
0.25
2.25
6.25


In [237]:
sum / 6

2.9166666666666665

In [245]:
(windows - means).T @ (windows - means) / 6

tensor([[2.9167, 2.9167, 2.9167, 2.9167, 2.9167],
        [2.9167, 2.9167, 2.9167, 2.9167, 2.9167],
        [2.9167, 2.9167, 2.9167, 2.9167, 2.9167],
        [2.9167, 2.9167, 2.9167, 2.9167, 2.9167],
        [2.9167, 2.9167, 2.9167, 2.9167, 2.9167]])